In [21]:
from simulated_bifurcation import Ising
from tqdm import tqdm
import matplotlib.pyplot as plt
import pulp as pl
import mosek
import numpy as np
import time
import os

N_range = [10,20,30,50,70,100,150,200]
scale_range = [6,5,4,3,2,1,0,-1]
N_samples = 1

def generate_J_h(N_range,scale_range,N_samples):
    for N in N_range:
        for scale in scale_range:
            for i in range(N_samples):
                J = np.random.uniform(-10,10,size=(N, N))/(10**scale)
                J = (J + J.T)/2
                J = J - np.diag(np.diag(J))
                h = np.random.uniform(-10,10,size=N)/(10**(scale-int(i%5)+2))

                with open('test_MOSEK/test_MOSEK_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, J)

                with open('test_MOSEK/test_MOSEK_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, h)

                with open('test_MOSEK/test_SB_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, J/4)

                with open('test_MOSEK/test_SB_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'wb') as f:
                    np.save(f, h/2 - sum([J[:,i]/8 for i in range(N)]) - sum([J[i,:]/8 for i in range(N)]))

generate_J_h(N_range,scale_range,N_samples)

In [22]:
def create_optimize_MOSEK(J,h,J_SB,h_SB,max_seconds=-1):
    time_1 = time.time()
    size = J.shape[0]

    model = pl.LpProblem("Linear Optimization", pl.LpMinimize)
    x = pl.LpVariable.dicts('x', list(range(size)), lowBound=0, upBound=1, cat='Integer')
    z = pl.LpVariable.dicts('z', [(i,j) for i in range(size) for j in range(size)], lowBound=0, upBound=1, cat='Integer')

    for i in range(size):
        for j in range(size):
            model += z[i,j] <= x[i]
            model += z[i,j] <= x[j]
            model += z[i,j] >= x[i] + x[j] -1

    model += pl.lpSum([(-1/2)*J[i][j]*z[i,j] for i in range(size) for j in range(size)]) + pl.lpSum([x[i]*h[i] for i in range(size)])

    if max_seconds == -1:
        model.solve(pl.MOSEK(msg=0, options={mosek.dparam.optimizer_max_time:10}))
        #model.solve(pl.PULP_MOSEK_CMD(msg=0, timeLimit=60*5))
    else:
        model.solve(pl.MOSEK(msg=0, options={mosek.dparam.optimizer_max_time:max_seconds}))

    spins = [1 if x[i].varValue >= 0.99 else -1 for i in range(size)]
    energy = (-1/2)*sum([sum([J_SB[i,j]*spins[j] for j in range(size)])*spins[i] for i in range(size)]) + sum([h_SB[i]*spins[i] for i in range(size)])[0]

    processing_time = time.time() - time_1
    return (processing_time,np.array(spins),energy)

In [23]:
def create_optimize_SB(J,h):
    time_1 = time.time()

    model = Ising(J,h)
    model.optimize(convergence_threshold = 50,
        sampling_period = 50,
        time_step = 0.01, 
        symplectic_parameter = 2,
        agents = 20,
        ballistic=True,
        heated=True,
        final_pressure=1.,
        pressure_slope=0.01,
        verbose=False)

    processing_time = time.time() - time_1
    return(processing_time, model.ground_state, model.energy)

In [24]:
def test_protocol(N_range,scale_range,N_samples,compute_SB=True):
    for N in tqdm(N_range):
        for scale in scale_range:
            for i in range(N_samples):
                with open('test_MOSEK/test_SB_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_SB_J = np.load(f)

                with open('test_MOSEK/test_SB_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_SB_h = np.expand_dims(np.load(f),axis=1)

                with open('test_MOSEK/test_MOSEK_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_MOSEK_J = np.load(f)

                with open('test_MOSEK/test_MOSEK_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy', 'rb') as f:
                    test_MOSEK_h = np.load(f)

                processing_time_MOSEK, spins_MOSEK, energy_MOSEK = create_optimize_MOSEK(test_MOSEK_J,test_MOSEK_h,test_SB_J,test_SB_h)
                with open('test_MOSEK/results_MOSEK_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'wb') as f:
                    np.savez(f, processing_time=processing_time_MOSEK, spins=spins_MOSEK, energy=energy_MOSEK, J=test_SB_J, h=test_SB_h)

                if compute_SB:
                    processing_time_sb, spins_sb, energy_sb = create_optimize_SB(test_SB_J,test_SB_h)
                    processing_time_MOSEK_stopped, spins_MOSEK_stopped, energy_MOSEK_stopped = create_optimize_MOSEK(test_MOSEK_J,test_MOSEK_h,test_SB_J,test_SB_h,processing_time_sb)

                    with open('test_MOSEK/results_SB_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'wb') as f:
                        np.savez(f, processing_time=processing_time_sb, spins=spins_sb, energy=energy_sb, J=test_SB_J, h=test_SB_h)

                    with open('test_MOSEK/results_MOSEKs_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'wb') as f:
                        np.savez(f, processing_time=processing_time_MOSEK_stopped, spins=spins_MOSEK_stopped, energy=energy_MOSEK_stopped, J=test_SB_J, h=test_SB_h)
                
                    os.remove('test_MOSEK/test_SB_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')
                    os.remove('test_MOSEK/test_SB_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')
                    os.remove('test_MOSEK/test_MOSEK_J_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')
                    os.remove('test_MOSEK/test_MOSEK_h_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npy')

In [25]:
test_protocol(N_range,scale_range,N_samples,compute_SB=True)

100%|██████████| 8/8 [12:48<00:00, 96.02s/it] 


In [27]:
results = {}

for N in tqdm(N_range):
    results[N] = {}
    for scale in scale_range:
        results[N][scale] = {}
        for i in range(N_samples):
            results[N][scale][i] = {}

            with open('test_MOSEK/results_MOSEK_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'rb') as f:
                data_MOSEK = np.load(f)
                results[N][scale][i]['MOSEK Energy'] = data_MOSEK['energy']
                results[N][scale][i]['MOSEK Spins'] = data_MOSEK['spins']
                results[N][scale][i]['MOSEK Time'] = data_MOSEK['processing_time']                

            with open('test_MOSEK/results_SB_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'rb') as f:
                data_SB = np.load(f)
                results[N][scale][i]['SB Energy'] = data_SB['energy']
                results[N][scale][i]['SB Spins'] = data_SB['spins']
                results[N][scale][i]['SB Time'] = data_SB['processing_time'] 

            with open('test_MOSEK/results_MOSEKs_' + str(N) + '_' + str(scale) + "_" + str(i) + '.npz', 'rb') as f:
                data_MOSEK_stopped = np.load(f)
                results[N][scale][i]['MOSEK Stopped Energy'] = data_MOSEK_stopped['energy']
                results[N][scale][i]['MOSEK Stopped Spins'] = data_MOSEK_stopped['spins']
                results[N][scale][i]['MOSEK Stopped Time'] = data_MOSEK_stopped['processing_time'] 

100%|██████████| 8/8 [00:00<00:00, 118.78it/s]


In [28]:
results

{10: {6: {0: {'MOSEK Energy': array(-4.77568855e-05),
    'MOSEK Spins': array([ 1, -1,  1, -1,  1,  1,  1,  1,  1, -1]),
    'MOSEK Time': array(0.00958276),
    'SB Energy': array(-4.77568855e-05),
    'SB Spins': array([[ 1.],
           [-1.],
           [ 1.],
           [-1.],
           [ 1.],
           [ 1.],
           [ 1.],
           [ 1.],
           [ 1.],
           [-1.]]),
    'SB Time': array(0.26647973),
    'MOSEK Stopped Energy': array(-4.77568855e-05),
    'MOSEK Stopped Spins': array([ 1, -1,  1, -1,  1,  1,  1,  1,  1, -1]),
    'MOSEK Stopped Time': array(0.00985193)}},
  5: {0: {'MOSEK Energy': array(-0.00061621),
    'MOSEK Spins': array([ 1,  1,  1,  1,  1, -1,  1,  1, -1,  1]),
    'MOSEK Time': array(0.00816917),
    'SB Energy': array(-0.00061621),
    'SB Spins': array([[ 1.],
           [ 1.],
           [ 1.],
           [ 1.],
           [ 1.],
           [-1.],
           [ 1.],
           [ 1.],
           [-1.],
           [ 1.]]),
    'SB Time': 